# Prova 1 - C210 - L3

### Instruções
* Esta prova contém 3 questões sobre, respectivamente, Machine Learning, Algoritmos Genéticos e PSO, cada uma pode possuir itens definidos por letras (a, b, c...)
* Esta prova deve ser feita INDIVIDUALMENTE, sem acesso a outros alunos
* É permitido o acesso aos códigos disponibilizados no Github da disciplina: https://github.com/TiagoMPereira/C210_IA
* Para a realização da prova apenas altere este notebook
* Ao final, envie na tarefa criada no Teams apenas o notebook com as alterações
* Preencha o campo a seguir com suas informações

#### Boa prova!!!

##### Nome: Isadora Bello Pereira Rodrigues
##### Curso: GEC
##### Matrícula: 1777

## 1) MACHINE LEARNING
###### Responda as seguintes questões com V (verdadeiro) ou F (falso). (Dê dois cliques sobre a célula para alterá-la)

a) O aprendizado supervisionado é um ramo de Machine Learning onde são fornecidos dados de treino e teste para que um algoritmo/modelo possa aprender. ( V )

b) Agrupamento é um tipo de aprendizado de máquina não supervisionado, no qual um algoritmo agrupa os dados de acordo com semelhanças entre eles. ( V )

c) Ao analisar diferentes modelos de aprendizado de máquina, é recomendado selecionar aquele que possua menor pontuação (R2_score) e maior erro. ( F )

d) Para prever o valor (real) de um imóvel, deve ser utilizado um modelo de regressão. ( V )

e) Para prever a presença ou ausência (1 ou 0) de determinada doença em uma pessoa deve ser utilizado um modelo de regressão. 
   ( F )

## 2) Algoritmos genéticos
###### Tendo como base o algoritmo apresentado abaixo, faça o que for pedido nas questões a), b) e c). Você pode executar a célula após fazer as questões se desejar, não é obrigatório. Não é necessário criar uma célula para cada questão, você pode alterar no próprio código disponibilizado

In [4]:
import numpy as np

In [25]:
############################################ BITSET ############################################
class BitSet:
    
    def __init__(self, size):
        self.bits = np.full(size, False)
        
    def __repr__(self):
        return (f"{self.bits*1}")
        
    def get(self, index): 
        return self.bits[index]
    
    def set_(self, index, value):
        self.bits[index] = value
        
    def flip(self, index):
        self.bits[index] = not self.bits[index]
        
    def debug(self):
        print(self.bits)
        
############################################ CROMOSSOMO ############################################
class Chromossome:
    
    def __init__(self, x = None, y = None):
        x = np.random.randint(-15, 16) if x is None else x
        y = np.random.randint(-15, 16) if y is None else y
        
        self.genes = Chromossome.get_genotype(x, y)
        self.score = np.inf
        
    def __repr__(self):
        return (f'(Gene: {self.genes}, Score: {self.score:.3f})')
    
    def __lt__(self, other):
        return self.score > other.score
    
    @staticmethod
    def to_binary(variable):
        variable = "{:05b}".format(variable)
        signal = 1 if variable[0] == "-" else 0
        variable = variable[1:]
        variable += str(signal)
        
        return variable

    @staticmethod
    def get_genotype(x, y):
        bits = BitSet(10)

        x_genotype = Chromossome.to_binary(x)
        y_genotype = Chromossome.to_binary(y)
        xy_binary = x_genotype + y_genotype
        
        for i in range(10):
            bits.set_(i, xy_binary[i] == '1')

        return bits

    @staticmethod
    def get_fenotype(genes):          
        
        x = (8 * genes.get(0) + 4 * genes.get(1) +
             2 * genes.get(2) + 1 * genes.get(3))

        if (genes.get(4) == 1):
            x *= -1

        y = (8 * genes.get(5) + 4 * genes.get(6) +
             2 * genes.get(7) + 1 * genes.get(8))

        if (genes.get(9) == 1):
            y *= -1

        return x, y
    
############################################ PROBLEMA ############################################
class Problem:

    @staticmethod
    def function(x, y):
        return 1 * (x**2) + 7 * (y**2)
    
    @staticmethod
    def score(x, y):
        return abs(Problem.function(x, y))

    @staticmethod
    def score_chromossome(chromossome):
        x, y = Chromossome.get_fenotype(chromossome.genes)
        return Problem.score(x, y)

    @staticmethod
    def score_average(population):
        avg = 0
        for chromossome in population:
            avg += Problem.score_chromossome(chromossome)
        avg /= len(population)
        return avg
    
    @staticmethod
    def update_chromossome_score(chromossome):
        genes_aux = Chromossome.get_fenotype(chromossome.genes)
        x = genes_aux[0]
        y = genes_aux[1]
        chromossome.score = Problem.score(x,y)
        
############################################ UTILITÁRIOS ############################################
class GeneticUtils:
    
    @staticmethod
    def find_best_chromossome(population):
        population.sort()
        return population[-1]

    @staticmethod
    def find_worst_chromossome(population):
        population.sort()
        return population[0]

    @staticmethod
    def roulette(population):
        population_scores = [chromossome.score for chromossome in population]
        population_scores = np.array(population_scores)

        # Calculando aptidão
        fitness = 1 / (population_scores + 1)
        total_fitness = fitness.sum()

        roulette_values = fitness / total_fitness
        return roulette_values

In [16]:
############################################ OPERADORES GENÉTICOS ############################################
class GeneticOperators:

    @staticmethod
    def selection(population):
        parents = np.random.choice(population, size=3, replace = False)
        population.sort()
        parent1, parent2 = parents[-1], parents[-2]
        
        population.sort()

        return parent1, parent2

    @staticmethod
    def selection_roulette(population):
        probabilities = GeneticUtils.roulette(population)
        parents = np.random.choice(population, size=2, replace = False, p = probabilities)
        parent1, parent2 = parents[0], parents[1]

        return parent1, parent2
    
    @staticmethod
    def crossover(population, parent1, parent2):
        crossover_point = np.random.randint(1, 10)
        parent1_genes = parent1.genes
        parent2_genes = parent2.genes

        child1_genes = BitSet(10)
        child2_genes = BitSet(10)

        for i in range(crossover_point):
            child1_genes.set_(i, parent1_genes.get(i))
            child2_genes.set_(i, parent2_genes.get(i))

        for i in range(crossover_point, 10):
            child1_genes.set_(i, parent2_genes.get(i))
            child2_genes.set_(i, parent1_genes.get(i))

        child1 = Chromossome()
        child1.genes = child1_genes
        
        child2 = Chromossome()
        child2.genes = child2_genes
        
        Problem.update_chromossome_score(child1)
        Problem.update_chromossome_score(child2)
        
        population.append(child1)
        population.append(child2)
        
    @staticmethod
    def mutation(population, mutation_threshold):
        ran = np.random.randint(1, 11)
        prob = np.random.uniform(0, 1)

        if prob >= mutation_threshold:
            return
        
        for i in range(ran):
            chromossome = np.random.choice(population)
            mutation_point = np.random.randint(0, 10)
            chromossome.genes.flip(mutation_point)
            Problem.update_chromossome_score(chromossome)
        
    @staticmethod
    def elitism(population):
        for _ in range(2):
            worst_individual = GeneticUtils.find_worst_chromossome(population)
            population.remove(worst_individual)

In [ ]:
############################################ PRINCIPAL ############################################
MAX_GENERATION = 100
INITIAL_POPULATION = 100
MUTATION_THRESHOLD = 0.01

if __name__ == "__main__":
    
    population = []
    for _ in range(INITIAL_POPULATION):
        chromossome_aux = Chromossome()
        Problem.update_chromossome_score(chromossome_aux)
        population.append(chromossome_aux)
        
    generation = 0
    
    while generation < MAX_GENERATION:
        
        parent1, parent2 = GeneticOperators.selection(population)
        
        GeneticOperators.crossover(population, parent1, parent2)

        GeneticOperators.mutation(population, MUTATION_THRESHOLD)

        GeneticOperators.elitism(population)
        
        generation += 1
        
    best_chromossome = GeneticUtils.find_best_chromossome(population)
    print(f"Best chromossome {best_chromossome}")

##### A) Na classe Problema, altere a função objetivo para: a * $x^2$ + b * $y^2$ , onde 'a' deve ser o primeiro dígito da sua matrícula e 'b' o último digito da sua matrícula.
##### Ex: Se sua matrícula for 123, 'a'  será 1 e 'b' será 3, então sua função objetivo deve ser  1 * $x^2$ + 3 * $y^2$ 

___

##### B) Altere a seleção dos pais de modo a selecionar 3 pais de forma aleatória (sem utilizar a roleta viciada) e, dentre os 3 pais selecionados, utilizar apenas os dois melhores para gerar novos filhos

___

##### C) Para cada geração, caso ocorra a mutação, sorteie um valor aleatório entre 1 e 10 (incluindo 10) e aplique a mutação na quantidade de cromossomos sorteadas

___

## 3) PSO
##### Na implementação do algoritmo The Cornfield Vector são considerados 3 vetores para calcular a nova velocidade das partículas. A soma dos vetores é feita de acordo com um peso definido para cada um. Para cada alternativa, altere os pesos dos vetores de modo que um seja priorizado em relação aos demais.
##### Observações:
* Nenhuma taxa deve ser menor ou igual a 0
* A soma das 3 taxas deve ser igual a 1 


* Considere a seguinte função para ajuste de velocidade

In [ ]:
def update_velocity(self, gbest_pos, velocity_rate):
    
    cognitive_velocity = self.calculate_velocity(self.pbest)
    
    social_velocity = self.calculate_velocity(gbest_pos)

    self.velocity = (np.array(self.velocity) * velocity_rate[0]
                    + cognitive_velocity * velocity_rate[1] 
                    + social_velocity * velocity_rate[2])

##### A) Altere a lista de taxa de velocidades de modo que priorize (seja maior que as demais) a velocidade atual da partícula ao invés das velocidades referentes ao Pbest e ao Gbest. 

VELOCITY_RATE = [0.8, 0.1, 0.1]

##### B) Altere a lista de taxa de velocidades de modo que priorize (seja maior que as demais) a velocidade referente ao Pbest da partícula ao invés da velocidade atual e da velocidade referente ao Gbest. 

VELOCITY_RATE = [0.1, 0.8, 0.1]

##### C) Altere a lista de taxa de velocidades de modo que priorize (seja maior que as demais) a velocidade referente ao Gbest da partícula ao invés da velocidade atual e da velocidade referente ao Pbest. 

VELOCITY_RATE = [0.1, 0.1, 0.8]